In [ ]:
import sys                                                                                                                                                                                                              
sys.path[0]='/home/jovyan/mnt/neuronunit'                                                                                                                                                                                                     
from neuronunit.tests import get_neab                                                                                                                                                                                                         
import neuronunit                                                                                                                                                                                                                             
print(neuronunit.tests.__file__)                                                                                                                                                                                                              
from neuronunit.tests import utilities as outils                                                                                                                                                                                              
                                                                                                                                                                                                                                              
from neuronunit.tests import model_parameters as modelp                                                                                                                                                                                       
import numpy as np                                                                                                                                                                                                                            
model = outils.model                
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
/home/jovyan/mnt/neuronunit/neuronunit/tests/__init__.py
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 2
        gap.NGEN = 2
        # The population of genes is 4
        gap.MU = 4
        
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]

        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        some_tuples = (vmpop, pop, invalid_ind, pf)
        return pop[0], some_tuples
        #return vmpop, pop, invalid_ind, pf
        
    def _get_optimization_parameters(self, some_tuples):
        vmpop = some_tuples[0]
        # Your specific unpacking of tuples that _optimize returns
        scores = [ i.score for i in vmpop ]
        pop = some_tuples[1]
        errors = pop[0].fitness
        parameters = vmpop[0].attrs
        #parameters,errors,scores,_ = zip(*some_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        The Class users version of optimize
        where details are hidden in _optimizae
        '''

        # Do optimization including repeated calls to judge
        models, some_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(some_tuples)
        # Maybe rebuild the original model
        # (i.e. restore the true class from the virtual version)

        # Your code keeps parameter sets and associated scores
        # All the organizing stuff
        
        
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.

        
        # Make a ScoreArray (which is basically a pandas dataframe)
       
        return model, scores, some_tuples

    
t = Test()


model,scores,some_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/neuronunit/neuronunit/tests/model_parameters.py'>
0 35.8798390412
1 -57.7247414243
2 0.000101810444432
3 -35.4038537857
4 -4.92543721595e-09
5 0.534438209282
6 0.126517561325
7 -58.2256604723
8 -49.5430386436
9 0.00139299232484
0 34.7669766553
1 -72.4680141235
2 0.000103201838209
3 -39.9945749613
4 -4.03805929367e-09
5 0.22691988156
6 0.139773835721
7 -56.134690759
8 -70.1714588654
9 0.00128519410792
0 35.81353006
1 -56.778105245
2 0.000108056805379
3 -48.6652811883
4 -3.52389055825e-09
5 0.836363715371


/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


6 0.0637532126724
7 -58.2470418091
8 -62.8825572622
9 0.00102812325425
0 33.2762178708
1 -66.9252339228
2 9.09649182952e-05
3 -41.351483459
4 -3.60377404106e-09
5 0.152613718942
6 0.0659157225379
7 -58.0524647919
8 -48.2935729089
9 0.000955975744271
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
<class 'float'>
current 173.61111111111111 spikes 1
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 338.96604938271605 spikes 1
{}
<class 'float'>
current 339.04320987654324 spikes 1
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 17.12962962962963 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 156.48148148148147 spikes 1
{}
{}
{}
<class 'float'>
checkpoint 1 output

{}
<class 'sciunit.scores.ZScore'>


{}


...

{}


{}


{}


{}


0 35.81353006
1 -56.778105245
2 0.000108056805379
3 -48.6652811883
4 -3.52389055825e-09
5 0.836363715371
6 0.0637532126724
7 -58.2470418091
8 -62.8825572622
9 0.00102812325425
0 35.8798390412
1 -57.7247414243
2 0.000101810444432
3 -35.4038537857
4 -4.92543721595e-09
5 0.534438209282
6 0.126517561325
7 -58.2256604723
8 -49.5430386436
9 0.00139299232484
0 33.2762178708
1 -66.9252339228
2 9.09649182952e-05
3 -41.351483459
4 -3.60377404106e-09
5 0.152613718942
6 0.0659157225379
7 -58.0524647919
8 -48.2935729089
9 0.000955975744271
0 34.7669766553
1 -72.4680141235
2 0.000103201838209
3 -39.9945749613
4 -4.03805929367e-09
5 0.22691988156
6 0.139773835721
7 -56.134690759
8 -70.1714588654
9 0.00128519410792
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 17.12962962962963 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}


In [ ]:
import sciunit
modelsp = [model.__class__(p) for p in parameters]
models = some_tuples[1]

path = sciunit.ScoreArray(models,scores=scores)

    

In [ ]:

import pickle
import pandas as pd


try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))
ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

#Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = some_tupeles[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])



In [ ]:


df_0

In [ ]:
df_1
